In [30]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号
import scipy.optimize as solver
import datetime as dt

#设置python 调用stata 17
import sys
sys.path.append(r'/Applications/Stata/utilities')  # Stata17的pystata包的安装位置
from pystata import config
config.init('mp')  # Stata17的版本

from pystata import stata
from glob import glob
from sfi import Data


In [31]:
df = pd.read_excel("WDI.xlsx",sheet_name="Data")
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,87.512260,88.129881,87.275323,88.720097,89.308602,90.283638,89.286856,NaN,NaN,NaN
2,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,77.251714,78.165706,75.512153,78.211000,79.065508,81.102134,79.248100,NaN,NaN,NaN
3,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,96.435957,96.772853,96.466705,96.936319,97.290083,97.467915,97.063959,NaN,NaN,NaN
4,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30.277130,NaN,NaN,37.165211,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380948,Zimbabwe,ZWE,Women who were first married by age 15 (% of w...,SP.M15.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.700000,NaN,NaN,NaN,5.4,NaN,NaN
380949,Zimbabwe,ZWE,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,33.500000,32.400000,NaN,NaN,NaN,33.7,NaN,NaN
380950,Zimbabwe,ZWE,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,59.200000,59.300000,59.400000,59.400000,59.500000,59.500000,59.600000,59.6,NaN,NaN
380951,Zimbabwe,ZWE,Young people (ages 15-24) newly infected with HIV,SH.HIV.INCD.YG,NaN,NaN,NaN,NaN,NaN,NaN,...,21000.000000,20000.000000,19000.000000,18000.000000,16000.000000,15000.000000,13000.000000,13000.0,NaN,NaN


In [32]:
del df["Unnamed: 65"]

In [33]:
df = df.iloc[:-1]
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,87.199474,87.512260,88.129881,87.275323,88.720097,89.308602,90.283638,89.286856,NaN,NaN
2,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,75.958878,77.251714,78.165706,75.512153,78.211000,79.065508,81.102134,79.248100,NaN,NaN
3,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,96.466418,96.435957,96.772853,96.466705,96.936319,97.290083,97.467915,97.063959,NaN,NaN
4,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,22.260538,NaN,NaN,30.277130,NaN,NaN,37.165211,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380947,Zimbabwe,ZWE,Women who believe a husband is justified in be...,SG.VAW.REFU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.900000,NaN,NaN,NaN,14.500000,NaN,NaN,NaN,NaN,NaN
380948,Zimbabwe,ZWE,Women who were first married by age 15 (% of w...,SP.M15.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.900000,NaN,NaN,NaN,3.700000,NaN,NaN,NaN,5.4,NaN
380949,Zimbabwe,ZWE,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,30.500000,NaN,NaN,33.500000,32.400000,NaN,NaN,NaN,33.7,NaN
380950,Zimbabwe,ZWE,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,59.000000,59.200000,59.300000,59.400000,59.400000,59.500000,59.500000,59.600000,59.6,NaN


In [34]:
del df["Country Name"]
del df["Indicator Name"]

In [35]:
df.columns.name = "Year"

In [36]:
df = df.set_index(["Country Code","Indicator Code"])
df

Year                            1960  1961  1962  1963  1964  1965  1966  \
Country Code Indicator Code                                                
ARB          EG.CFT.ACCS.ZS      NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             EG.ELC.ACCS.ZS      NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             EG.ELC.ACCS.RU.ZS   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             EG.ELC.ACCS.UR.ZS   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             FX.OWN.TOTL.ZS      NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...                              ...   ...   ...   ...   ...   ...   ...   
ZWE          SG.VAW.REFU.ZS      NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             SP.M15.2024.FE.ZS   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             SP.M18.2024.FE.ZS   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             SH.DYN.AIDS.FE.ZS   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
             SH.HIV.INCD.YG      NaN   NaN   NaN   NaN   NaN   NaN   NaN   

Year                            1967  1968  1969  ...          2011  \
Country Code Indicator Code                       ...                 
ARB          EG.CFT.ACCS.ZS      NaN   NaN   NaN  ...     82.783289   
             EG.ELC.ACCS.ZS      NaN   NaN   NaN  ...     87.199474   
             EG.ELC.ACCS.RU.ZS   NaN   NaN   NaN  ...     75.958878   
             EG.ELC.ACCS.UR.ZS   NaN   NaN   NaN  ...     96.466418   
             FX.OWN.TOTL.ZS      NaN   NaN   NaN  ...     22.260538   
...                              ...   ...   ...  ...           ...   
ZWE          SG.VAW.REFU.ZS      NaN   NaN   NaN  ...     16.900000   
             SP.M15.2024.FE.ZS   NaN   NaN   NaN  ...      3.900000   
             SP.M18.2024.FE.ZS   NaN   NaN   NaN  ...     30.500000   
             SH.DYN.AIDS.FE.ZS   NaN   NaN   NaN  ...     59.000000   
             SH.HIV.INCD.YG      NaN   NaN   NaN  ...  23000.000000   

Year                                    2012          2013          2014  \
Country Code Indicator Code                                                
ARB          EG.CFT.ACCS.ZS        83.120303     83.533457     83.897596   
             EG.ELC.ACCS.ZS        87.512260     88.129881     87.275323   
             EG.ELC.ACCS.RU.ZS     77.251714     78.165706     75.512153   
             EG.ELC.ACCS.UR.ZS     96.435957     96.772853     96.466705   
             FX.OWN.TOTL.ZS              NaN           NaN     30.277130   
...                                      ...           ...           ...   
ZWE          SG.VAW.REFU.ZS              NaN           NaN           NaN   
             SP.M15.2024.FE.ZS           NaN           NaN           NaN   
             SP.M18.2024.FE.ZS           NaN           NaN     33.500000   
             SH.DYN.AIDS.FE.ZS     59.200000     59.300000     59.400000   
             SH.HIV.INCD.YG     21000.000000  20000.000000  19000.000000   

Year                                    2015          2016          2017  \
Country Code Indicator Code                                                
ARB          EG.CFT.ACCS.ZS        84.171599     84.510171           NaN   
             EG.ELC.ACCS.ZS        88.720097     89.308602     90.283638   
             EG.ELC.ACCS.RU.ZS     78.211000     79.065508     81.102134   
             EG.ELC.ACCS.UR.ZS     96.936319     97.290083     97.467915   
             FX.OWN.TOTL.ZS              NaN           NaN     37.165211   
...                                      ...           ...           ...   
ZWE          SG.VAW.REFU.ZS        14.500000           NaN           NaN   
             SP.M15.2024.FE.ZS      3.700000           NaN           NaN   
             SP.M18.2024.FE.ZS     32.400000           NaN           NaN   
             SH.DYN.AIDS.FE.ZS     59.400000     59.500000     59.500000   
             SH.HIV.INCD.YG     18000.000000  16000.000000  15000.000000   

Year                                    2018     2019  2020  
Country Code Indicator Code                    

In [37]:
df.unstack(level=1).stack(level=0)

Indicator Code     AG.AGR.TRAC.NO  AG.CON.FERT.PT.ZS  AG.CON.FERT.ZS  \
Country Code Year                                                      
ABW          1960             NaN                NaN             NaN   
             1961             NaN                NaN             NaN   
             1962             NaN                NaN             NaN   
             1963             NaN                NaN             NaN   
             1964             NaN                NaN             NaN   
...                           ...                ...             ...   
ZWE          2016             NaN         249.520154           32.50   
             2017             NaN         345.495495           38.35   
             2018             NaN         345.495495           38.35   
             2019             NaN                NaN             NaN   
             2020             NaN                NaN             NaN   

Indicator Code     AG.LND.AGRI.K2  AG.LND.AGRI.ZS  AG.LND.ARBL.HA  \
Country Code Year                                                   
ABW          1960             NaN             NaN             NaN   
             1961            20.0       11.111111          2000.0   
             1962            20.0       11.111111          2000.0   
             1963            20.0       11.111111          2000.0   
             1964            20.0       11.111111          2000.0   
...                           ...             ...             ...   
ZWE          2016        162000.0       41.876696       4000000.0   
             2017        162000.0       41.876696       4000000.0   
             2018        162000.0       41.876696       4000000.0   
             2019             NaN             NaN             NaN   
             2020             NaN             NaN             NaN   

Indicator Code     AG.LND.ARBL.HA.PC  AG.LND.ARBL.ZS  AG.LND.CREL.HA  \
Country Code Year                                                      
ABW          1960                NaN             NaN             NaN   
             1961           0.036076       11.111111             NaN   
             1962           0.035571       11.111111             NaN   
             1963           0.035276       11.111111             NaN   
             1964           0.035068       11.111111             NaN   
...                              ...             ...             ...   
ZWE          2016           0.285095       10.339925       1551550.0   
             2017           0.280963       10.339925       1438930.0   
             2018           0.277027       10.339925       1641701.0   
             2019                NaN             NaN             NaN   
             2020                NaN             NaN             NaN   

Indicator Code     AG.LND.CROP.ZS  ...  per_sa_allsa.cov_q4_tot  \
Country Code Year                  ...                            
ABW          1960             NaN  ...                      NaN   
             1961             NaN  ...                      NaN   
             1962             NaN  ...                      NaN   
             1963             NaN  ...                      NaN   
             1964             NaN  ...                      NaN   
...                           ...  ...                      ...   
ZWE          2016        0.258498  ...                      NaN   
             2017        0.258498  ...                 9.032539   
             2018        0.258498  ...                      NaN   
             2019             NaN  ...                18.893733   
             2020             NaN  ...                      NaN   

Indicator Code     per_sa_allsa.cov_q5_tot  per_si_allsi.adq_pop_tot  \
Country Code Year                                                      
ABW          1960                      NaN                       NaN   
             1961                      NaN                       NaN   
             1962                      NaN                       NaN

In [38]:
df = _

In [39]:
df

Indicator Code     AG.AGR.TRAC.NO  AG.CON.FERT.PT.ZS  AG.CON.FERT.ZS  \
Country Code Year                                                      
ABW          1960             NaN                NaN             NaN   
             1961             NaN                NaN             NaN   
             1962             NaN                NaN             NaN   
             1963             NaN                NaN             NaN   
             1964             NaN                NaN             NaN   
...                           ...                ...             ...   
ZWE          2016             NaN         249.520154           32.50   
             2017             NaN         345.495495           38.35   
             2018             NaN         345.495495           38.35   
             2019             NaN                NaN             NaN   
             2020             NaN                NaN             NaN   

Indicator Code     AG.LND.AGRI.K2  AG.LND.AGRI.ZS  AG.LND.ARBL.HA  \
Country Code Year                                                   
ABW          1960             NaN             NaN             NaN   
             1961            20.0       11.111111          2000.0   
             1962            20.0       11.111111          2000.0   
             1963            20.0       11.111111          2000.0   
             1964            20.0       11.111111          2000.0   
...                           ...             ...             ...   
ZWE          2016        162000.0       41.876696       4000000.0   
             2017        162000.0       41.876696       4000000.0   
             2018        162000.0       41.876696       4000000.0   
             2019             NaN             NaN             NaN   
             2020             NaN             NaN             NaN   

Indicator Code     AG.LND.ARBL.HA.PC  AG.LND.ARBL.ZS  AG.LND.CREL.HA  \
Country Code Year                                                      
ABW          1960                NaN             NaN             NaN   
             1961           0.036076       11.111111             NaN   
             1962           0.035571       11.111111             NaN   
             1963           0.035276       11.111111             NaN   
             1964           0.035068       11.111111             NaN   
...                              ...             ...             ...   
ZWE          2016           0.285095       10.339925       1551550.0   
             2017           0.280963       10.339925       1438930.0   
             2018           0.277027       10.339925       1641701.0   
             2019                NaN             NaN             NaN   
             2020                NaN             NaN             NaN   

Indicator Code     AG.LND.CROP.ZS  ...  per_sa_allsa.cov_q4_tot  \
Country Code Year                  ...                            
ABW          1960             NaN  ...                      NaN   
             1961             NaN  ...                      NaN   
             1962             NaN  ...                      NaN   
             1963             NaN  ...                      NaN   
             1964             NaN  ...                      NaN   
...                           ...  ...                      ...   
ZWE          2016        0.258498  ...                      NaN   
             2017        0.258498  ...                 9.032539   
             2018        0.258498  ...                      NaN   
             2019             NaN  ...                18.893733   
             2020             NaN  ...                      NaN   

Indicator Code     per_sa_allsa.cov_q5_tot  per_si_allsi.adq_pop_tot  \
Country Code Year                                                      
ABW          1960                      NaN                       NaN   
             1961                      NaN                       NaN   
             1962                      NaN                       NaN

In [40]:
df.columns

Index(['AG.AGR.TRAC.NO', 'AG.CON.FERT.PT.ZS', 'AG.CON.FERT.ZS',
       'AG.LND.AGRI.K2', 'AG.LND.AGRI.ZS', 'AG.LND.ARBL.HA',
       'AG.LND.ARBL.HA.PC', 'AG.LND.ARBL.ZS', 'AG.LND.CREL.HA',
       'AG.LND.CROP.ZS',
       ...
       'per_sa_allsa.cov_q4_tot', 'per_sa_allsa.cov_q5_tot',
       'per_si_allsi.adq_pop_tot', 'per_si_allsi.ben_q1_tot',
       'per_si_allsi.cov_pop_tot', 'per_si_allsi.cov_q1_tot',
       'per_si_allsi.cov_q2_tot', 'per_si_allsi.cov_q3_tot',
       'per_si_allsi.cov_q4_tot', 'per_si_allsi.cov_q5_tot'],
      dtype='object', name='Indicator Code', length=1443)

In [42]:
df.to_csv("data/WDI.csv")

In [50]:
!open "data"